In [1]:
from bs4 import BeautifulSoup
import re
import xmlToSql
import sqlite3
import pandas as pd

In [2]:
with open('frharrypotter_pages_current.xml','r') as f:
    file = f.read()

soup = BeautifulSoup(file, 'xml')

In [3]:
pages = soup.find_all('page')

In [ ]:
with open('personnages.txt','x') as f:
    for page in pages:
        if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
            f.write(xmlToSql.getAttribute("Nom",page.text) +" "+ xmlToSql.getAttribute("Yeux",page.text)+"\n")

In [17]:
def getAttribute(attribute, target):
    matchingString = f'\|{attribute}=\[*([\w\ ,\']*)\W.*?\s\|'
    result="?"
    if attribute == "Mort" or attribute == "Naissance":
        try:
            result = re.search('\|{}=.*?(\d{4}).*?\s\|',target).group(1)
        except:
            return "0"
    else:
        if attribute == "Sang":
            matchingString = "\|Sang=\[\[([^]]+)\]\]"
        try:
            result = re.search(matchingString, target).group(1)
            if result == "":
                result="?"
        except:
            return "?"
    result = result.replace("'","('')")
    return result

def getNom(target):
    try:
        result = re.search('\|Nom=(.*?)\s\|', target).group(1)
    except:
        return ""
    return result

def getYeux(target):
    try:
        result = re.search('\|Yeux=([\w\ 0-9,\']*).*?\s\|', target).group(1)
    except:
        return ""
    return result

def getGenre(target):
    try:
        result = re.search('\|Genre=\ ?([A-Za-z0-9â]*?)\s\|', target).group(1)
    except:
        return ""
    return result

def getCheveux(target):
    try:
        result = re.search('\|Cheveux=([A-Za-z0-9]*?)\s\|', target).group(1)
        if result == "":
            return "Inconnu"
    except:
        return "Inconnu"
    return result


def getNaissance(target):
    try:
        result = re.search('\|Naissance=.*?(\d{4}).*?\s\|',target).group(1)
    except:
        return "0"
    return result

def getMort(target):
    try:
        result = re.search('\|Mort=.*?(\d{4}).*?\s\|',target).group(1)
    except:
        return "0"
    return result


# with open('personnagesnaissance.txt','x') as f:
#     for page in pages:
#         if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
#             f.write(getAttribute("Nom",page.text) +" "+ getAttribute("Naissance",page.text)+"\n")
for page in pages[0:100]:
    if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
        id = ""
        for attr in ["Nom","Cheveux","Yeux","Genre","Patronus","Sang"]:
            id += f" {getAttribute(attr,page.text)}"
        id += str(getMort(page.text))
        print(id)

 Harry Potter Noirs Verts Homme Cerf Sang-Mêlé0
 Drago Malefoy Blonds Gris Homme ? Sang-Pur0
 Ronald Weasley Roux Bleus Homme Jack Russell terrier Sang-Pur0
 Albus Dumbledore Argentés Bleus Homme Phénix Sang-Mêlé1997
 Tom Elvis Jedusor Bruns puis chauve Marron puis rouges Homme Aucun, incapable d('')avoir une pensée heureuse Sang-Mêlé1998
 Alastor Maugrey Gris Un œil petit et sombre et un autre grand, rond, d('')un bleu électrique Homme ? Sorcier1997
 Minerva McGonagall Noirs Verts Femme Chat Sang-Mêlé0


## Creating the database


In [13]:
connection = sqlite3.connect("harrypotter.db")
cursor = connection.cursor()

In [14]:
cursor.execute("CREATE TABLE personnages (nom TEXT NOT NULL PRIMARY KEY, genre TEXT, yeux TEXT, cheveux TEXT, naissance INTEGER, mort INTEGER, patronus TEXT, sang TEXT)")

In [19]:
for page in pages:
    if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
        nom = getAttribute("Nom",page.text)
        yeux = getAttribute("Yeux",page.text)
        cheveux = getAttribute("Cheveux",page.text)
        genre = getAttribute("Genre",page.text)
        patronus = getAttribute("Patronus",page.text)
        naissance = getAttribute("Naissance",page.text)
        mort = getAttribute("Mort",page.text)
        sang = getAttribute("Sang",page.text)
        print(f"Insert INTO personnages VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
        try:
            cursor.execute(f"Insert INTO personnages VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
        except:
            print(f"Already inserted {nom}")

Insert INTO personnages VALUES('Harry Potter','Homme','Verts','Noirs','0','0','Cerf','Sang-Mêlé')
Already inserted Harry Potter
Insert INTO personnages VALUES('Drago Malefoy','Homme','Gris','Blonds','0','0','?','Sang-Pur')
Already inserted Drago Malefoy
Insert INTO personnages VALUES('Ronald Weasley','Homme','Bleus','Roux','0','0','Jack Russell terrier','Sang-Pur')
Already inserted Ronald Weasley
Insert INTO personnages VALUES('Albus Dumbledore','Homme','Bleus','Argentés','0','0','Phénix','Sang-Mêlé')
Already inserted Albus Dumbledore
Insert INTO personnages VALUES('Tom Elvis Jedusor','Homme','Marron puis rouges','Bruns puis chauve','0','0','Aucun, incapable d('')avoir une pensée heureuse','Sang-Mêlé')
Insert INTO personnages VALUES('Alastor Maugrey','Homme','Un œil petit et sombre et un autre grand, rond, d('')un bleu électrique','Gris','0','0','?','Sorcier')
Insert INTO personnages VALUES('Minerva McGonagall','Femme','Verts','Noirs','0','0','Chat','Sang-Mêlé')
Insert INTO personnages

Insert INTO personnages VALUES('Zacharias Loreyon','Homme','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('Daisy Pennifold','Femme','?','?','0','0','?','Sorcière')
Insert INTO personnages VALUES('Nouye Empot','Homme','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('Kevin Broadmoor','Homme','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('Karl Broadmoor','Homme','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('Glynnis Griffiths ','Femme','?','?','0','0','?','Sorcière')
Insert INTO personnages VALUES('Gwendolyn Morgan ','Femme','?','?','0','0','?','Sorcière')
Insert INTO personnages VALUES('Rudolf Brand ','Homme','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('Darren O('')Hare','Homme','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('Eunice Murray ','Femme','?','?','0','0','?','Sorcière')
Insert INTO personnages VALUES('Catriona McCormack','Femme','?','?','0','0','?','Sorcière')
Insert INTO personnages 

Insert INTO personnages VALUES('W','?','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('I','?','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('B','?','?','?','0','0','?','Sorcier')
Already inserted B
Insert INTO personnages VALUES('J','?','?','?','0','0','?','Sorcier')
Already inserted J
Insert INTO personnages VALUES('P','?','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('Élève de Gryffondor qui possède un Boulurlard','Femme','?','Noirs','0','0','?','Sorcière')
Insert INTO personnages VALUES('Smyth','?','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('A','?','?','?','0','0','?','Sorcier')
Already inserted A
Insert INTO personnages VALUES('G','?','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('P','?','?','?','0','0','?','Sorcier')
Already inserted P
Insert INTO personnages VALUES('MacFarlan','?','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('Protheroe','?','?','?','0','0','?','Sorcier')
Insert INTO

Insert INTO personnages VALUES('Père de Chadwick et Webster Boot','Homme','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('Mère de Chadwick et Webster Boot','Femme','?','?','0','0','?','Sorcière')
Insert INTO personnages VALUES('Corvus Lestrange I','Homme','?','?','0','0','?','Sang-Pur')
Insert INTO personnages VALUES('Héloïse Lestrange','Femme','?','?','0','0','?','Sang-Pur')
Insert INTO personnages VALUES('Corvus Lestrange II','Homme','?','?','0','0','?','Sang-Pur')
Insert INTO personnages VALUES('Minette Rosier','Femme','?','?','0','0','?','Sang-Pur')
Insert INTO personnages VALUES('Manon Lestrange','Femme','?','?','0','0','?','Sang-Pur')
Insert INTO personnages VALUES('Josette Lestrange','Femme','?','?','0','0','?','Sang-Pur')
Insert INTO personnages VALUES('Corvus Lestrange III','?','?','?','0','0','?','Sang-Pur')
Insert INTO personnages VALUES('Églantine Lestrange','Femme','?','?','0','0','?','Sang-Pur')
Insert INTO personnages VALUES('Corvus Lestrange IV','Homme','

Insert INTO personnages VALUES('Luca Caruso ','Homme','?','?','0','0','?','?')
Insert INTO personnages VALUES('Mandy Brocklehurst','Femme','?','?','0','0','?','?')
Insert INTO personnages VALUES('Xander Lofthouse','Homme','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('Mère de Vernon et Marjorie Dursley','Femme','?','?','0','0','?','Moldu')
Insert INTO personnages VALUES('Enfants de Dudley Dursley','?','?','?','0','0','?','Moldu')
Insert INTO personnages VALUES('McDuff','Homme','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('McClaggan','Homme','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('Achille Tolliver','Homme','?','?','0','0','?','Sorcier')
Insert INTO personnages VALUES('Shafiq','Homme','?','?','0','0','?','Sang-Pur')
Insert INTO personnages VALUES('Yaxley','Homme','?','?','0','0','?','Sang-Pur')
Insert INTO personnages VALUES('Helixa Hyslop','Femme','?','?','0','0','?','Sorcière')
Insert INTO personnages VALUES('Syna Hyslop','Femme',

In [20]:
connection.commit()
cursor.close()
connection.close()

### Now table for créatures

In [ ]:
cursor.execute("CREATE TABLE personnages (nom TEXT NOT NULL PRIMARY KEY, genre TEXT, yeux TEXT, cheveux TEXT, naissance INTEGER, mort INTEGER, patronus TEXT, sang TEXT)")